In [5]:
import sys; sys.path.append('../3rdparty/ElasticRods/python')
import elastic_rods, elastic_knots
import numpy as np, matplotlib.pyplot as plt, time, io, os

from helpers import *
from parametric_curves import *
import py_newton_optimizer

from linkage_vis import LinkageViewer as Viewer, CenterlineViewer
from tri_mesh_viewer import PointCloudViewer, PointCloudMesh

%load_ext autoreload
%autoreload 2

import parallelism
parallelism.set_max_num_tbb_threads(1)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
file = '../data/L400-r0.2-UpTo9Crossings/4_1/0001.obj'
rod_radius = 0.2
material = elastic_rods.RodMaterial('ellipse', 2000, 1, [rod_radius, rod_radius])
centerline = read_nodes_from_file(file)  # supported formats: obj, txt
pr = define_periodic_rod(centerline, material)
rod_list = elastic_knots.PeriodicRodList([pr])
len(rod_list.getDoFs())

1601

In [7]:
view = Viewer(rod_list, width=1024, height=800)
view.show()

Renderer(camera=PerspectiveCamera(aspect=1.28, children=(PointLight(color='#999999', position=(0.0, 0.0, 5.0),…

In [8]:
def callback(problem, iteration):
    if iteration % 5 == 0:
        view.update()
for i in range(1,10):
    print(f"iterration: {i}")
    optimizerOptions = py_newton_optimizer.NewtonOptimizerOptions()
    optimizerOptions.niter = 1000
    optimizerOptions.gradTol = 1e-8
    hessianShift = 1e-4 * compute_min_eigenval_straight_rod(pr)

    problemOptions = elastic_knots.ContactProblemOptions()
    problemOptions.contactStiffness = 1e-3
    problemOptions.dHat = 2*rod_radius *i
    fixedVars = []   
    
    report = elastic_knots.compute_equilibrium(
        rod_list, problemOptions, optimizerOptions, 
        fixedVars=fixedVars,
        externalForces=np.zeros(rod_list.numDoF()),
        softConstraints=[],
        callback=callback,
        hessianShift=hessianShift
        )
    view.update()

iterration: 1
0	0.798935	0.0310317	0.0310317	0.0234375	1
1	0.797836	0.0302637	0.0302637	1	1
2	0.797495	0.00513874	0.00513874	1	1
3	0.797462	0.000195485	0.000195485	1	1
4	0.79743	0.000201085	0.000201085	1	1
5	0.797418	9.09041e-05	9.09041e-05	1	1
6	0.797396	0.000126718	0.000126718	1	1
7	0.797385	9.00269e-05	9.00269e-05	1	1
8	0.797317	0.00044484	0.00044484	1	1
9	0.79717	0.00156319	0.00156319	0.5	0
10	0.797099	0.00949138	0.00949138	1	0
11	0.79704	0.00159146	0.00159146	0.25	0
12	0.797037	0.00254309	0.00254309	1	0
13	0.797028	0.000809519	0.000809519	1	0
14	0.797026	0.00171762	0.00171762	1	0
15	0.797025	0.000223574	0.000223574	1	0
16	0.797024	0.000735113	0.000735113	1	0
17	0.797024	5.24514e-05	5.24514e-05	1	0
18	0.797024	3.83775e-05	3.83775e-05	1	0
19	0.797024	6.375e-08	6.375e-08	1	0
20	0.797024	2.79522e-11	2.79522e-11	1	0
iterration: 2
0	0.840131	0.0988638	0.0988638	1	1
1	0.83399	0.701359	0.701359	1	1
2	0.817947	0.386456	0.386456	1	0
3	0.805957	0.206311	0.206311	1	1
4	0.803244	0.0164389	0.01

The simulations will continue, but the result might be non-physical due to spurious contact forces between neighboring edges.
Consider decreasing the cross-section radius or using a coarser polyline.
Increasing the minContactEdgeDist parameter would remove non-physical contact forces, too, but only at the expense of topology preservation guarantees.



0	0.861311	0.10481	0.10481	1	1
1	0.814865	0.0674042	0.0674042	1	1
2	0.809611	0.0111229	0.0111229	1	1
3	0.807976	0.00687675	0.00687675	1	1
4	0.807194	0.00470071	0.00470071	1	1
5	0.806767	0.00313775	0.00313775	1	0
6	0.806412	0.0622038	0.0622038	1	0
7	0.805882	0.000472243	0.000472243	1	0
8	0.805874	0.000890246	0.000890246	1	0
9	0.805874	1.2009e-06	1.2009e-06	1	0
10	0.805874	1.08862e-07	1.08862e-07	1	0
11	0.805874	8.51855e-12	8.51855e-12	1	0
iterration: 4


The simulations will continue, but the result might be non-physical due to spurious contact forces between neighboring edges.
Consider decreasing the cross-section radius or using a coarser polyline.
Increasing the minContactEdgeDist parameter would remove non-physical contact forces, too, but only at the expense of topology preservation guarantees.



0	0.893209	0.159261	0.159261	1	1
1	0.831467	0.0982551	0.0982551	1	1
2	0.818493	0.0183957	0.0183957	1	1
3	0.814401	0.0103206	0.0103206	1	1
4	0.81259	0.0068002	0.0068002	1	1
5	0.811604	0.00459493	0.00459493	1	1
6	0.811019	0.00335745	0.00335745	1	0
7	0.810447	0.0598401	0.0598401	1	0
8	0.81003	0.00019145	0.00019145	1	0
9	0.810027	0.00030302	0.00030302	1	0
10	0.810027	3.2508e-08	3.2508e-08	1	0
11	0.810027	8.28204e-12	8.28204e-12	1	0
iterration: 5


...

0	4.94756	0.3176	0.3176	1	1
1	4.82379	0.129496	0.129496	1	1
2	4.80275	0.0267387	0.0267387	1	1
3	4.79774	0.0121479	0.0121479	1	1
4	4.79586	0.00849421	0.00849421	1	1
5	4.79487	0.00595413	0.00595413	1	1
6	4.79434	0.00386244	0.00386244	1	0
7	4.79379	0.0429944	0.0429944	1	0
8	4.79356	3.90665e-05	3.90665e-05	1	0
9	4.79356	6.39611e-05	6.39611e-05	1	0
10	4.79356	2.03066e-08	2.03066e-08	1	0
11	4.79356	7.71599e-12	7.71599e-12	1	0
iterration: 8


The simulations will continue, but the result might be non-physical due to spurious contact forces between neighboring edges.
Consider decreasing the cross-section radius or using a coarser polyline.
Increasing the minContactEdgeDist parameter would remove non-physical contact forces, too, but only at the expense of topology preservation guarantees.



0	15.7194	0.387185	0.387185	1	1
1	15.5706	0.139366	0.139366	1	1
2	15.5466	0.0303616	0.0303616	1	1
3	15.5412	0.0126546	0.0126546	1	1
4	15.5393	0.00848078	0.00848078	1	1
5	15.5382	0.00598213	0.00598213	1	1
6	15.5377	0.00400522	0.00400522	1	0
7	15.5372	0.0385954	0.0385954	1	0
8	15.537	4.6085e-05	4.6085e-05	1	0
9	15.537	7.51987e-05	7.51987e-05	1	0
10	15.537	4.31941e-09	4.31941e-09	1	0
iterration: 9


The simulations will continue, but the result might be non-physical due to spurious contact forces between neighboring edges.
Consider decreasing the cross-section radius or using a coarser polyline.
Increasing the minContactEdgeDist parameter would remove non-physical contact forces, too, but only at the expense of topology preservation guarantees.



0	41.017	0.489951	0.489951	1	1
1	40.8156	0.146623	0.146623	1	1
2	40.7896	0.0330483	0.0330483	1	1
3	40.7841	0.0123631	0.0123631	1	1
4	40.782	0.00877865	0.00877865	1	1
5	40.781	0.00591747	0.00591747	1	1
6	40.7805	0.00402033	0.00402033	1	0
7	40.78	0.0368901	0.0368901	1	0
8	40.7798	2.99851e-05	2.99851e-05	1	0
9	40.7798	3.57473e-05	3.57473e-05	1	0
10	40.7798	1.98779e-09	1.98779e-09	1	0


In [10]:
from helpers import write_obj
file = '../data/NoCollision/0001.obj'
write_obj(file, rod_list)